# Setup and configuration
This notebook walks through the actual code in this document for Power BI [Multi-tenancy](https://learn.microsoft.com/en-us/power-bi/developer/embedded/embed-multi-tenancy)

Steps needed to run this notebook:
1. Create a app registration as stated in this [link](https://learn.microsoft.com/en-us/power-bi/developer/embedded/embed-service-principal#step-1---create-an-azure-ad-app)
    > 💡Copy the Domain/Tenant ID, Application/Client ID and Token created
1. From a Power BI tenant admin account, enable creating profiles in the tenant using the same security group you used when you created the service principal.
1. Create a `.env` in this folder with the following variables:
    ```
    TenantId=<guid of your tenant in Azure>
    ClientId=<App reg id you create>
    ClientSecret=<Secret for the app you create>
    sqlUser=<sql user name>
    sqlPwd=<sql password>
    ```
    - This is optional if you don't care about updating the SQL credentials. This will at least show you how to do it. There is one table with a list of customers that are Fortune 500 by state

## Assumptions
 You know how to create and work a Python Notebook which includes setting up an environment and installing the required modules:
 ```
azure-identity==1.12.0
pandas
python-dotenv
 ```

# Import the required modules

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd, json
from azure.identity import DefaultAzureCredential,ClientSecretCredential
import requests

Authenticate


In [ ]:
api = 'https://analysis.windows.net/powerbi/api/.default'

load_dotenv()

credit_auth = True
#This loads the environment variables from the .env file
if credit_auth:
    tenant_id = os.getenv('TenantId')
    client_id = os.getenv('ClientId')
    client_secret = os.getenv('ClientSecret')
    auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                                                        tenant_id = tenant_id,
                                                        client_id = client_id,
                                                        client_secret = client_secret)
    access_token = auth.get_token(api)   
else:
    # This assumes the keys are in the environment variables file AZURE_TENANT_ID,AZURE_CLIENT_ID and AZURE_CLIENT_SECRET
    azure_credential = DefaultAzureCredential() 
    access_token = azure_credential.get_token(api) 

base_url = 'https://api.powerbi.com/v1.0/myorg/'
header = {'Authorization': f'Bearer {access_token.token}'}

In [ ]:
#Variables for the workspace and report. None of these should exist at the beginning of the script
profileName="profile2"
workspaceName="Test for Meeting"

#Have a local PBIX file ready to publish. I have included a sample one
filePath='/path to a pbix file/Customer.pbix'
reportName="Customer"

## Functions

In [ ]:
print(os.getcwd())

# function to upload a file to power bi
def upload_file(headers, groupId, fileName,reportName):
    url = 'https://api.powerbi.com/v1.0/myorg/groups/' + groupId + '/imports?nameConflict=Abort&datasetDisplayName=' + reportName

    # create a copy of the headers
    headers_copy = headers.copy()
    headers_copy['Content-Type'] = 'multipart/form-data'
    files = {
        'file': (reportName+'.pbix', open(fileName, 'rb')),
    }

    r = requests.post(
        url=url,
        files=files,
        headers=headers_copy
    )


## Get Profile Id

In [ ]:
# Get a list of profiles
response = requests.get(base_url + 'profiles', headers=header).json()
# print(f"Count={response['@odata.count']}")
print(base_url + 'profiles')
print(response)


## Create the profile


In [ ]:
body = {
    "displayName": profileName,
}
response = requests.post(base_url + 'profiles', headers=header, json=body).json()
profileId=response['id']

# add the profile id to the header
header['X-PowerBI-Profile-Id']=profileId

## List Workspaces


In [ ]:
groups = requests.get(base_url + f"groups", headers=header).json()

print(json.dumps(groups, indent=4, sort_keys=True))

In [ ]:
# Create a new workspace
body = {
    "name": workspaceName
}
response = requests.post(base_url + 'groups', headers=header, json=body).json()
workspaceId=response['id']
print(json.dumps(response, indent=4, sort_keys=True))

## Upload a PBIX file

In [ ]:
upload_file(header, workspaceId, filePath,reportName)

In [ ]:
# Get the imports for the workspace
imports = requests.get(base_url + "groups/" + workspaceId + "/imports", headers=header).json()
print(json.dumps(imports, indent=4, sort_keys=True))

Get the dataset and update any of the properties for the UserName/Password and Parameter

In [ ]:
dataset= requests.get(base_url + "groups/" + workspaceId + "/datasets", headers=header).json()
print(json.dumps(dataset, indent=4))

for val in dataset['value']:
    dataSources= requests.get(base_url + "groups/" + workspaceId + "/datasets/" + val['id'] + "/datasources", headers=header).json()
    print(json.dumps(dataSources, indent=4))
    for v in dataSources['value']:
        if v['datasourceType'] == 'Sql':
            creds={
                "credentialData":[
                        {"name":"username","value":os.getenv('sqlUser')},
                        {"name":"password","value":os.getenv('sqlPwd')}
                ]
            }

            body={
                "credentialDetails":json.dumps(creds),
                "credentialType":"Basic",
                "encryptedConnection":"Encrypted",
                "encryptionAlgorithm":"None",
                "privacyLevel":"None"
            }
            
            response = requests.post(base_url + "gateways/" + v["gatewayId"] + "/datasources/" + v['datasourceId'] + "/Default.UpdateDatasource", headers=header, json=body)
            print("Updating dataset",base_url + "gateways/" + v["gatewayId"] + "/datasources/" + v['datasourceId'] + "/Default.UpdateDatasource")
            print(response)



In [ ]:
#Update parameters
# https://docs.microsoft.com/en-us/rest/api/power-bi/datasets/updateparametersingroup
# https://docs.microsoft.com/en-us/rest/api/power-bi/datasets/getparametersingroup
response = requests.get(base_url + "groups/" + workspaceId + "/datasets", headers=header).json()
print(json.dumps(response, indent=4))
for val in response['value']:
    body={
        "updateDetails": [
            {
                "name": "StateCode",
                "newValue": "AR"
            }
        ]
    }
    response = requests.post(base_url + "groups/" + workspaceId + "/datasets/" + val['id'] + "/Default.UpdateParameters", headers=header, json=body)
    print(response)

In [ ]:
#Refresh the datasets
# https://docs.microsoft.com/en-us/rest/api/power-bi/datasets/refreshdatasetingroup
response = requests.get(base_url + "groups/" + workspaceId + "/datasets", headers=header).json()
print(json.dumps(response, indent=4))
for val in response['value']:
    response = requests.post(base_url + "groups/" + workspaceId + "/datasets/" + val['id'] + "/refreshes", headers=header)
    print(response)

In [ ]:
#Get the last 5 refreshes
# https://docs.microsoft.com/en-us/rest/api/power-bi/refreshhistories/getrefreshhistoriesingroup
response = requests.get(base_url + "groups/" + workspaceId + "/datasets", headers=header).json()
# print(json.dumps(response, indent=4))
for val in response['value']:
    response = requests.get(base_url + "groups/" + workspaceId + "/datasets/" + val['id'] + "/refreshes?$top=5", headers=header).json()
    print(json.dumps(response, indent=4))

## Generate an Embed Token
This is what would happen in a JavaScript, TypeScript call

In [ ]:
# Get a power bi embed token
# https://learn.microsoft.com/en-us/rest/api/power-bi/embed-token/generate-token
response = requests.get(base_url + "groups/" + workspaceId + "/reports", headers=header).json()
print(json.dumps(response, indent=4))
reportId=response['value'][0]['id']
datasetId=response['value'][0]['datasetId']

body = {
    "datasets": [
        {
            "id": datasetId
        }
    ],
    "reports": [
        {
            "id": reportId
        }
    ]
}
response = requests.post(base_url + "GenerateToken", headers=header, json=body).json()
print(json.dumps(response, indent=4))